# Optimisation des paramètres de simulation avec Hyperopt

**Contexte:**
    
Dans ce notebook, nous cherchons à produire des Time-Series de simulation qui se rapproche un maximum des Time-Series de réference.
Pour ce faire, nous utilisons la méthode d'optimisation Hyperopt pour trouver la combinaison de paramètres pour le modèle simplifié qui se rapproche du modèle de réference.

Hyperopt prend l'espace de recherche que l'on lui fournit, utilisé un algorithme de suggestion pour décider des paramètres qu'il souhaite évaluer et enfin Hyperopt utilise une fonction objectif et cherche à minimiser cette fonction objectif.

Les fonctions objectifs utilisable avec Hyperopt sont les fonctions objectifs qui sont minimisables

## Importation des données

Nous importons les dépendances nécessaires pour faire fonctionner le notebook

In [1]:

import sys
sys.path.append("..")
import yaml
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from hyperopt import hp, fmin, tpe

from uvsw_part.simulation import run_cable_wakeosc

import matplotlib.pyplot as plt
import math

## Définition du chemin vers le fichier Liste*.txt et de le numéro de la Time-Series de réference utilisé

Le chemin vers le fichier Liste* permet de récuperer les paramètres de base des Time-Series de référence, l'index permet de choisir sur quelle Time-series nous allons travailler (commence à 0)

In [2]:
# Path to list of time series
LIST_PATH = "../data/params/List1.txt"
# Index of time series to simulate
TS_INDEX = 5

## Chargement de la Time-Series de reférence

Ici, nous chargeons le fichier Liste utilisé ainsi que la Time-Series de réference auquel nous allons comparé la simulation

In [3]:
# Recupération du numéro de fichier Liste
LIST_REF = LIST_PATH[-9:-4].lower()

# Read table of time series
data_list = pd.read_csv(LIST_PATH , delim_whitespace=True)
# Get parameters of selected time series
set_params = data_list.iloc[TS_INDEX,:]
# Get dataframe of data of corresponding TS

#Chargement de la time-series de référence
ref = pd.read_csv("../data/ref/"+ LIST_REF + "/graph{}.csv".format(set_params["nc"]))

In [4]:
# Show params of selected TS
set_params

nc                    6
md                    1
U[m/s]            1.265
d[m]              0.025
m[kg/m]            1.57
L[m]                  1
H[N]                500
Nt                10000
Dt[s]            0.0004
tf[s]               4.0
ymax[m]        0.000003
filename    Data1.6.bin
Name: 5, dtype: object

In [5]:
#Définition de paramètres permettant d'obtenir des Time-Series de simulation de même longueur que les Time-Series de réference
tf_val = set_params["tf[s]"]
dt_val = tf_val / len(ref)
dr_val = tf_val / len(ref)

## Définition de l'espace de recherche d'Hyperopt

Ici nous définissons les plages de paramètres qui seront explorées par Hyperopt afin de trouver une combinaison de paramètres pour la simulation qui se rapproche de la réference

In [6]:
search_space = {
    'cable': {
        'type': 'None',
        'length': 1.0,
        'tension': hp.uniform('tension_val',100.0,40000.0),
        'h': hp.uniform('h_val',0.0,1000.0)},
    'conductor': {'m': 1.57, 'd': 0.025, 'EA': 0.0},
    'simulation': {'ns': 101,
        'tf': tf_val,
        'dt': dt_val,
        'dr': dr_val,
        'si': 99,
        'pp': False}, ####### Enlever les prints
    'wakeosc': {'u': hp.uniform('u_val',0.05,7.00),
        'st': 0.235,
        'cl0': 0.6,
        'eps': 0.3,
        'al': 0.1,
        'bt': 0.0,
        'gm': 0.0,
        'md': 1,
        'y0': 0.0,
        'q0': 0.05}
}

## Calcul du taux de couverture

Nous calculons une métrique permettant de connaître le taux de valeurs issus simulation comprise à l'intérieur de la time-series de réference pour chaque timesteps. Nous comparons le signe de la valeur issu de la réference et de la reference pour chaque temps, et nous comparons si la simulation a une amplitude absolue inférieur à celle de réference.

In [12]:
def t_couverture (ref,dfy):
    """
    Vérification si les valeurs de reference et simulation ont le même signe un par un
    Si même signe, opérations appropriées pour voir si l'amplitude de la simulation négative ou positive est bien contenue dans la reference
    Arguments :
    ref (pandas.core.series.Series) (numpy.ndarray) : Modèle de reférence
    dfy (numpy.ndarray)                             : Modèle simplifié genéré par le simulateur
    
    
    Sortie :
        Taux de couverture des données (float): Pourcentage de valeurs de la simulation 
        à l'intérieur de la réference   
    """
    same_sign = 0 # Nombres de même signe dans reference et simulation (pris 1 par 1)
    diff_sign = 0 # Nombres de signes différents dans reference et simulation (pris 1 par 1)
    couverture = 0 # Nombres de même signe ET avec la ref qui couvre la simulation
    
    ref_taille = len(ref) # Nombre total de valeurs dans ref et sim
    
    
    for i in range(0,ref_taille):
        if(np.sign(ref[i]) == np.sign(dfy[i])): #Si la reference et la sim sont du même signe
            same_sign = same_sign + 1


            if(ref[i] < 0): # Quand la reference < 0
                if(ref[i] - dfy[i] < 0):
                    couverture = couverture + 1
                

            if(ref[i] > 0): # Quand la reference > 0
                if(ref[i] - dfy[i] > 0):
                    couverture = couverture + 1

            if(ref[i] == 0): # Quand la reference = 0
                if(ref[i] - dfy[i] == 0):
                    couverture = couverture + 1

            if(math.isnan(ref[i]) and math.isnan(dfy[i]) == True): # Quand la reference = NaN
                couverture = couverture + 1             
            
        else :
            diff_sign = diff_sign + 1 # Si ce n'est pas le même signe
            

    taux_couverture = (couverture / ref_taille) * 100 # Calcul du taux en pourcentage
    return taux_couverture

## Définition des fonctions objectifs utilisés

Ici, nous définissons les fonctions objectifs qui sont utilisés avec Hyperopt dans l'objectif de récuperer des combinaison de paramètres se rapprochant du modèle de réference.
Les fonctions objectifs calculent des métriques qui sont utilisés pour évaluer la similarité entre la Time-Series de simulation et celle de réference.
Les métriques obtenu doivent être minimisés par Hyperopt

## Fonction Objectif : Root Mean Squared Error (RMSE)

La fontion objectif utilisé ici utilise une métrique basé sur le RMSE

In [13]:
def rmse_sim(cfg):
    """

    Fonction permettant de calculer une fonction objectif basé sur le RMSE et d'afficher les graphes pour chaque itération d'hyperopt
    
    Arguments : Espace de recherche (search_space)
    
    Sortie : Score (float) :calculé avec la fonction coût (RMSE)
             Plot avec les valeurs proposées par Hyperopt et métriques
    
    """
    # Run simulation
    dfy, _ = run_cable_wakeosc(cfg)
    print("yes")
    # Current config
    print("#"*100)
    print("Current parameters:")
    print("\tU value: {}, \n\tH value: {}, \n\tTension value: {}, \n\tcl0 value: {}, \n\teps value: {}".format(
        cfg['wakeosc']['u'],
        cfg['cable']['h'],
        cfg['cable']['tension'],
        cfg['wakeosc']['cl0'],
        cfg['wakeosc']['eps'])
    )
    print("#"*100)
    
    # Calculate score
    score = np.sqrt(mean_squared_error(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1]))
    
    # Score saved in string for the plot
    score_text = "RMSE = %s " % score

    # Plot graphics
    plt.figure(figsize = (20,5))
    plt.plot(ref['time'], ref['y/d'], label = "Signal de reference")
    plt.plot(dfy.index, dfy['s=0.250']/0.025, label = "Signal du simulateur")
    plt.xlabel('time (s)')
    plt.ylabel('y/d')
    
    # Plot score and parameters
    plt.figtext(0.5, -0.05, score_text, ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
    plt.figtext(0.5, -0.20, "h = '{0}', tension = '{1}', u = '{2}', clo = '{3}', eps = '{4}', st = '{5}' ".format(cfg['cable']['h'],
                                                                                                                  cfg['cable']['tension'],cfg['wakeosc']['u'],cfg["wakeosc"]["cl0"],
                                                                                                                  cfg["wakeosc"]["eps"],cfg["wakeosc"]["st"]), ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
    title = "Comparaison du signal de simulation avec les valeurs de paramètres fournis par HYPEROPT avec le signal de reference, Fonction objectif utilisée : RMSE"
    plt.title(title,fontsize=18)


    plt.legend()
    plt.show() 

    
    print("#"*40)
    print("Current scores:")
    print("\tR2 : {}".format(r2_score(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])))
    print("\tMSE : {}".format(mean_squared_error(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])))
    print("#"*40)
    # Delete the simulation
    del dfy
    # Return the score
    return score

## Fonction objectif : basé sur le R2 SCORE

La fonction objectif définie ici est une fonction basé sur le R2_SCORE. Afin d'avoir une formule qui peut être minimisé par Hyperopt, la formule de calcul est définie par 1.0 - R2_SCORE

In [14]:
def r2_sim(cfg):
    
    """

    Fonction permettant de calculer la fonction coût et d'afficher les graphes pour chaque itération d'hyperopt 
    Arguments : Espace de recherche (search_space)
    
    Sortie : Score calculé avec la fonction coût (1 - R2_score)
            Plot avec les valeurs proposées par Hyperopt et métriques
    
    """
    # Run simulation
    dfy, _ = run_cable_wakeosc(cfg)
    print("yes")
    # Current config
    print("#"*100)
    print("Current parameters:")
    print("\tU value: {}, \n\tH value: {}, \n\tTension value: {}, \n\tcl0 value: {}, \n\teps value: {}".format(
        cfg['wakeosc']['u'],
        cfg['cable']['h'],
        cfg['cable']['tension'],
        cfg['wakeosc']['cl0'],
        cfg['wakeosc']['eps'])
    )
    print("#"*100)
    
    # Calculate score
    score = 1.0 - r2_score(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])
    taux_couverture = t_couverture(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])
    r2 = r2_score(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])
    
    # Store score in a string for visualisation purpose
    score_text = "Score (1 - R2) = %s " % score
    r2_text = "R2 = %s " % r2
    taux_couverture_text = "Taux de couverture = %s %%" % taux_couverture
    
    plt.figure(figsize = (20,5))
    plt.plot(ref['time'], ref['y/d'], label = "Signal de reference")
    plt.plot(dfy.index, dfy['s=0.250']/0.025, label = "Signal du simulateur")
    plt.xlabel('time (s)')
    plt.ylabel('y/d')
    
    # Plot score_value and parameters value
    plt.figtext(0.5, -0.05, score_text, ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
    plt.figtext(0.5, -0.20, r2_text, ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})

    plt.figtext(0.5, -0.30, "h = '{0}', tension = '{1}', u = '{2}', clo = '{3}', eps = '{4}', st = '{5}' ".format(cfg['cable']['h'],
                                                                                                        cfg['cable']['tension'],cfg['wakeosc']['u'],cfg["wakeosc"]["cl0"],
                                                                                                                  cfg["wakeosc"]["eps"],cfg["wakeosc"]["st"]), ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
    plt.figtext(0.5, -0.40, taux_couverture_text, ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})

    title = "Comparaison du signal de simulation avec les valeurs de paramètres fournis par HYPEROPT avec le signal de reference, Fonction objectif utilisée : R2_score"
    plt.title(title,fontsize=18)


    plt.legend()
    plt.show() 
    # Compute the score

    
    print("#"*40)
    print("Current scores:")
    print("\tR2 : {}".format(r2_score(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])))
    print("\tMSE : {}".format(mean_squared_error(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])))
    print("#"*40)
    # Delete the simulation
    del dfy
    # Return the score
    return score

## Lancement du processus d'optimisation avec Hyperopt

Nous lançons l'algorithme Hyperopt avec la fonction objectif choisie, l'espace de recherche souhaité, l'algorithme de suggestion utilisé par Hyperopt et le nombre maximum d'itérations

In [ ]:
# Run the optimization process
best = fmin(r2_sim, search_space, algo=tpe.suggest, max_evals=500)

  0%|          | 0/500 [00:00<?, ?trial/s, best loss=?]